# Nafisur Rahman
nafisur21@gmail.com<br>
https://www.linkedin.com/in/nafisur-rahman

### Finding most probable answer from wikipedia article

In [5]:
import urllib
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
from nltk import pos_tag
import numpy as np
import pandas as pd
import sklearn
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
import re



In [6]:
def getTextfromurl(url):
    page = urllib.request.urlopen(url).read().decode('utf8')
    soup = BeautifulSoup(page,"lxml")
    text = ' '.join(map(lambda p: p.text, soup.findAll('p')[0:5]))
    return text.encode('ascii', errors='replace').decode('utf-8').replace("?"," ")


In [7]:
articleURL ='https://en.wikipedia.org/wiki/Kerala'
text1=getTextfromurl(articleURL)
text1

"Kerala (/ k r l /), historically known as Keralam, is a state in South India on the Malabar Coast. It was formed on 1 November 1956 following the States Reorganisation Act by combining Malayalam-speaking regions. Spread over 38,863 km2 (15,005 sq mi), it is bordered by Karnataka to the north and northeast, Tamil Nadu to the east and south, and the Lakshadweep Sea to the west. With 33,387,677 inhabitants as per the 2011 Census, Kerala is the thirteenth-largest Indian state by population. It is divided into 14 districts with the capital and the largest city being Thiruvananthapuram. Malayalam is the most widely spoken language and is also the official language of the state. The Chera Dynasty was the first prominent kingdom based in Kerala. The Ay kingdom in the deep south and the Ezhimala kingdom in the north formed the other kingdoms in the early years of the Common Era (CE or AD). The region had been a prominent spice exporter since 3000 BCE. The region's prominence in trade was noted

In [9]:
def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None
    
def doc_to_synsets(doc):
    text=str(doc)
    text=re.sub('[^a-zA-Z]',' ',text)
    text=word_tokenize(text)
    #print(text)
    #text=[w for w in text if w not in Cstopwords]
    #text=[stemmer.stem(w) for w in text if w not in Cstopwords]
    #text=[lemma.lemmatize(w) for w in text if w not in Cstopwords]
    tags=pos_tag(text)
    #print(tags)
    synset_list=[]
    for tag in tags:
        try:
            ss=wn.synsets(tag[0],pos=convert_tag(tag[1]))[0]
            synset_list.append(ss)
        except:
            None
    return synset_list

In [10]:
def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    
    list_of_largest_similarities = []
    # Your Code Here
    for syn1 in s1:
        
        largest_sim = 0
        
        for syn2 in s2:
            
            #print(syn1.path_similarity(syn2))
            
            if syn1.path_similarity(syn2):
            
                if syn1.path_similarity(syn2) > largest_sim:
                
                    largest_sim = syn1.path_similarity(syn2)
                
        list_of_largest_similarities.append(largest_sim)
    
    return sum(list_of_largest_similarities) / (len(list_of_largest_similarities)+1)


In [11]:
def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [14]:
def similar(question,text, n):
    sents = sent_tokenize(text)
    
    assert n <= len(sents)
    
    
    ranking = defaultdict(int)
    
    for i,sent in enumerate(sents):
        score=document_path_similarity(question, sent)
        ranking[i] += score
             
        
    sents_idx = nlargest(n, ranking, key=ranking.get)
    return [sents[j] for j in (sents_idx)]

In [16]:
question='most widely spoken language in Kerala?'

similar(question,text1,2)

['Malayalam is the most widely spoken language and is also the official language of the state.',
 'The state has the highest media exposure in India with newspapers publishing in nine languages, mainly English and Malayalam.']

In [17]:
doc1='first prominent kingdom based in Kerala'
similar(doc1,text1,2)

['The Chera Dynasty was the first prominent kingdom based in Kerala.',
 'The Ay kingdom in the deep south and the Ezhimala kingdom in the north formed the other kingdoms in the early years of the Common Era (CE or AD).']